In [ ]:
import torch
import numpy as np
import torchvision
from torchvision import datasets 
import torchvision.transforms  as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader , random_split
from torch import nn 
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'gpu'  ) 
torch.manual_seed(13)

In [ ]:
path= './DATASET/'
device
min_batch = 512
TRAIN_SIZE = 50000
VAL_SIZE = 5000
TEST_SIZE = 5000

In [ ]:
transform_cifar10_train = T.Compose([
                T.RandomHorizontalFlip(p=0.3),
                T.ColorJitter(brightness=0.1, contrast=0.1, hue = 0.05),
                T.RandomApply([T.RandomRotation(10), T.Resize(40), T.CenterCrop(32)], p = 0.1),
                T.ToTensor(),
                T.Normalize([0.491, 0.482, 0.447], [0.247, 0.243, 0.262])
            ])
transform_cifar10_test = T.Compose([
                T.ToTensor(),
                T.Normalize([0.491, 0.482, 0.447], [0.247, 0.243, 0.262])
            ])

In [ ]:
cifar10_train = datasets.CIFAR10(path, train=True, download=False,transform=transform_cifar10_train)


test_dataset = datasets.CIFAR10(path, train=False, download=False, transform=transform_cifar10_test)
val_dataset, test_dataset = random_split(test_dataset, [VAL_SIZE, TEST_SIZE])

val_loader = DataLoader(val_dataset, batch_size=min_batch, shuffle = True)
train_loader = DataLoader(cifar10_train, batch_size=min_batch, shuffle = True)

In [ ]:
# for i,(x, y) in enumerate(train_loader):
#     print(i, x.shape, y.shape)

In [ ]:
def plot_cifar10_grid():
    classes = train_loader.dataset.classes
    total_samples = 8
    plt.figure(figsize=(15,15))
    for label, sample in enumerate(classes):
        class_idxs = np.flatnonzero(label == np.array(train_loader.dataset.targets))
        sample_idxs = np.random.choice(class_idxs, total_samples, replace = False)
        for i, idx in enumerate(sample_idxs):
            plt_idx = i*len(classes) + label + 1
            plt.subplot(total_samples, len(classes), plt_idx)
            plt.imshow(train_loader.dataset.data[idx])
            plt.axis('off')
            
            if i == 0: plt.title(sample)
    plt.show()

plot_cifar10_grid() 

# ResNet

In [ ]:
def conlayer_K3P1(canal_in , canal_out , stride):
    return nn.Conv2d(canal_in , canal_out , stride=stride , kernel_size=3 , padding=1)

In [ ]:
class bloque_residual(nn.Module):
    def __init__(self , in_channel , out_channel , stride=1 , change_size = True):
        super().__init__() 
        ## CNN
        self.conv1= conlayer_K3P1(in_channel , out_channel , stride )
        self.bn1= nn.BatchNorm2d(out_channel)
        self.conv2 = conlayer_K3P1(out_channel , out_channel , 1 )
        self.bn2= nn.BatchNorm2d(out_channel)
        # Para cambiar el map size : 
        self.change_size = change_size
        if change_size : 
            self.residual = nn.Sequential(
                
                nn.Conv2d(in_channel ,
                                    out_channel,
                                    kernel_size=1 ,
                                    stride=stride),
                nn.BatchNorm2d(out_channel)
                
            )
        
    def forward(self, x ): 
        if not self.change_size:
            residual_in= x  
        else:
            residual_in= x=self.residual(x)
    
        y = F.ReLU(self.bn1(self.conv1(x)))
        y = self.bn2(self.conv2(y))
        y = residual_in + y 
        return F.relu(y)
            
        

In [ ]:
# n sirve para iterar la creacion de codigo y no tener que escribir N capas a mano

class Resnet56(nn.Module):
    def __init__(self , n=9 , num_classes=10):
        super().__init__()
        
        self.conv1 = conlayer_K3P1(3,16 ,stride=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.block1 = self.crete_block(n=9 , 
                                       in_channel=16 ,
                                       out_channel=16 ,
                                       stride=1 ,
                                       change_size= False)
       
        self.block2 = self.crete_block(n=9 , in_channel=16 , 
                                       out_channel=32,
                                       stride=2,
                                       change_size=True)
        
        self.block3 = self.crete_block(n=9, in_channel=32,
                                       out_channel=64,
                                       stride=2,)
        
        self.avg_pool = nn.AdaptiveAvgPool2d
        
        self.fc = nn.Linear(64 , num_classes)
        
        
    def crete_block (self , n , in_channel , out_channel , stride , change_size=True ):
        block = [bloque_residual(in_channel , out_channel , stride , change_size )]
        for i in range(n-1):
            block.append(bloque_residual( out_channel , out_channel , stride=1 , change_size=False ))
        return nn.Sequential(*block)
    
    def forward(self , x):
        
        y = F.relu(self.bn1(self.conv1(x)))
        y = self.block1(y)
        y = self.block2(y)
        y = self.avg_pool(self.block3(y))
        
        return self.fc(y.view(y.size(0) , -1))
        
        

In [ ]:
model = Resnet56()
optimizer_resnet56 = torch.optim.SGD(model.parameters() , lr= 0.1 , momentum=0.95 , weight_decay=1e-4)

# Train 

